In [37]:
import tensorflow as tf
from tensorflow.keras import models,layers

In [38]:
import matplotlib.pyplot as plt
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS =3
EPOCHS = 30

In [39]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
"PlantVillage",
shuffle = True,
image_size = (IMAGE_SIZE,IMAGE_SIZE),
batch_size = BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [40]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [41]:
len(dataset)

68

In [42]:
# for image_batch, label_batch in dataset.take(1):
#   print ("hi")    
train_ds = dataset.take(54)
len(train_ds)

54

In [43]:
test_ds = dataset.skip(54)
len(test_ds)

14

In [44]:
val_ds = test_ds.take(6)
len(val_ds)

6

In [45]:
test_ds = test_ds.skip(6)
len(test_ds)

8

In [46]:
def get_dataset_partitions_tf (ds, train_split=0.8, val_split=0.1, test_split =0.1, shuffle= True, shuffle_size = 10000):
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
        
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [47]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size= tf.data.AUTOTUNE)

In [48]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255) 
 ])

In [49]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [50]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE , CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale ,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation ='relu', input_shape = input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size =(3,3) ,activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size =(3,3) ,activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3) ,activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3) ,activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3) ,activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape = input_shape)

In [51]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 sequential_4 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (32, 127, 127, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                  

In [52]:
model.compile(
   optimizer = 'adam',
   loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
   metrics = ['accuracy'])

In [53]:
history = model.fit(
   train_ds,
   epochs = EPOCHS,
   batch_size = BATCH_SIZE,
   verbose = 1,
   validation_data = val_ds
)

Epoch 1/30
54/54 [==============================] - 59s 1s/step - loss: 0.9306 - accuracy: 0.4560 - val_loss: 0.8415 - val_accuracy: 0.4948
Epoch 2/30
54/54 [==============================] - 59s 1s/step - loss: 0.8781 - accuracy: 0.5897 - val_loss: 0.6632 - val_accuracy: 0.7188
Epoch 3/30
54/54 [==============================] - 59s 1s/step - loss: 0.6125 - accuracy: 0.7431 - val_loss: 0.7709 - val_accuracy: 0.6875
Epoch 4/30
54/54 [==============================] - 58s 1s/step - loss: 0.5106 - accuracy: 0.7876 - val_loss: 0.3843 - val_accuracy: 0.8594
Epoch 5/30
54/54 [==============================] - 65s 1s/step - loss: 0.3899 - accuracy: 0.8362 - val_loss: 0.2957 - val_accuracy: 0.8802
Epoch 6/30
54/54 [==============================] - 55s 1s/step - loss: 0.3053 - accuracy: 0.8785 - val_loss: 0.2604 - val_accuracy: 0.9010
Epoch 7/30
54/54 [==============================] - 35s 654ms/step - loss: 0.2312 - accuracy: 0.9057 - val_loss: 0.3647 - val_accuracy: 0.8594
Epoch 8/30
54/54 

In [54]:
scores = model.evaluate(test_ds)

8/8 [==============================] - 3s 236ms/step - loss: 0.1289 - accuracy: 0.9612


In [55]:
scores

[0.1289280205965042, 0.9612069129943848]

In [56]:
history

In [57]:
history.params

{'verbose': 1, 'epochs': 30, 'steps': 54}

In [58]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [59]:
import numpy as np

In [60]:
for images_batch, labels_batch in test_ds.take(1):
    first_label = labels_batch[0].numpy()
    print("actual_label:" , class_names[first_label])
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

actual_label: Potato___Early_blight
1/1 [==============================] - 0s 296ms/step
predicted label: Potato___Early_blight


In [61]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])),2)
    return predicted_class, confidence

In [62]:
# plt.figure(figsize(15,15))
for images, labels in test_ds.take(1):
    for i in range(9):
#         ax = plt.subplot(3,3,i+1)
#         plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]
        
        print(f"Actual: {actual_class},\n Predicted:{predicted_class}, \n Confidence: {confidence}%")
#         plt.axis("off")

1/1 [==============================] - 0s 51ms/step
Actual: Potato___Late_blight,
 Predicted:Potato___Late_blight, 
 Confidence: 99.58%
1/1 [==============================] - 0s 41ms/step
Actual: Potato___Early_blight,
 Predicted:Potato___Early_blight, 
 Confidence: 93.39%
1/1 [==============================] - 0s 51ms/step
Actual: Potato___Late_blight,
 Predicted:Potato___Late_blight, 
 Confidence: 99.68%
1/1 [==============================] - 0s 48ms/step
Actual: Potato___Late_blight,
 Predicted:Potato___Late_blight, 
 Confidence: 100.0%
1/1 [==============================] - 0s 64ms/step
Actual: Potato___Late_blight,
 Predicted:Potato___Late_blight, 
 Confidence: 99.88%
1/1 [==============================] - 0s 48ms/step
Actual: Potato___Early_blight,
 Predicted:Potato___Early_blight, 
 Confidence: 100.0%
1/1 [==============================] - 0s 48ms/step
Actual: Potato___Late_blight,
 Predicted:Potato___Late_blight, 
 Confidence: 99.9%
1/1 [==============================] - 0s 72m

In [63]:
import os 
# model_version=0
model_version = max([int(i) for i in os.listdir("../models")+[0]])+1
model.save(f"../models/{model_version}")

INFO:tensorflow:Assets written to: ../models/3\assets


INFO:tensorflow:Assets written to: ../models/3\assets
